In [1]:
from collections import deque
import random
import time
import numpy as np

class MultiStartTabuSearch:
    def __init__(self, distance_matrix, num_restarts=10, tabu_size=30, max_iterations=500):
        self.distance_matrix = distance_matrix
        self.n = len(distance_matrix)
        self.num_restarts = num_restarts
        self.tabu_size = tabu_size
        self.max_iterations = max_iterations
    
    def random_solution(self):
        """Génère une solution aléatoire"""
        solution = list(range(self.n))
        random.shuffle(solution)
        return solution + [solution[0]]
    
    def solution_cost(self, solution):
        """Calcule le coût d'une solution"""
        total = 0
        for i in range(len(solution) - 1):
            total += self.distance_matrix[solution[i]][solution[i+1]]
        return total
    
    def generate_neighbors_2opt(self, solution):
        """Génère les voisins par 2-opt"""
        neighbors = []
        n = len(solution)
        for i in range(1, n-3):
            for j in range(i+2, n-1):
                neighbor = solution.copy()
                neighbor[i:j+1] = reversed(neighbor[i:j+1])
                neighbors.append(neighbor)
        return neighbors
    
    def get_move_signature(self, old_sol, new_sol):
        """Signature pour la liste tabou"""
        for i in range(len(old_sol)):
            if old_sol[i] != new_sol[i]:
                start = i
                break
        for i in range(len(old_sol)-1, -1, -1):
            if old_sol[i] != new_sol[i]:
                end = i
                break
        return ('2opt', start, end)
    
    def single_tabu_search(self, initial_solution):
        """Une recherche Tabou simple"""
        current_sol = initial_solution
        current_cost = self.solution_cost(current_sol)
        best_sol = current_sol.copy()
        best_cost = current_cost
        
        tabu_list = deque(maxlen=self.tabu_size)
        iteration = 0
        
        while iteration < self.max_iterations:
            iteration += 1
            
            neighbors = self.generate_neighbors_2opt(current_sol)
            best_neighbor = None
            best_neighbor_cost = float('inf')
            best_move = None
            
            for neighbor in neighbors:
                neighbor_cost = self.solution_cost(neighbor)
                move = self.get_move_signature(current_sol, neighbor)
                
                # Critère d'aspiration
                if neighbor_cost < best_cost:
                    best_neighbor = neighbor
                    best_neighbor_cost = neighbor_cost
                    best_move = move
                    break
                
                # Meilleur non-tabou
                if move not in tabu_list and neighbor_cost < best_neighbor_cost:
                    best_neighbor = neighbor
                    best_neighbor_cost = neighbor_cost
                    best_move = move
            
            if best_neighbor is None:
                break
                
            current_sol = best_neighbor
            current_cost = best_neighbor_cost
            
            if current_cost < best_cost:
                best_sol = current_sol.copy()
                best_cost = current_cost
            
            if best_move:
                tabu_list.append(best_move)
        
        return best_sol, best_cost
    
    def solve(self):
        """MULTI-START TABU SEARCH - Tout en un !"""
        best_global_sol = None
        best_global_cost = float('inf')
        all_results = []
        
        print(f"🚀 MULTI-START TABU SEARCH - {self.num_restarts} relances\n")
        start_time = time.time()
        
        for restart in range(self.num_restarts):
            # Solution initiale aléatoire
            initial_sol = self.random_solution()
            initial_cost = self.solution_cost(initial_sol)
            
            # Recherche Tabou
            best_sol, best_cost = self.single_tabu_search(initial_sol)
            
            improvement = initial_cost - best_cost
            all_results.append({
                'restart': restart + 1,
                'initial_cost': initial_cost,
                'final_cost': best_cost,
                'improvement': improvement
            })
            
            print(f"Relance {restart+1:2d}: {initial_cost:8.2f} → {best_cost:8.2f} (Δ: {improvement:7.2f})")
            
            if best_cost < best_global_cost:
                best_global_sol = best_sol
                best_global_cost = best_cost
        
        # Analyse des résultats
        self.analyze_results(all_results, best_global_cost, time.time() - start_time)
        
        return best_global_sol, best_global_cost

    def analyze_results(self, all_results, best_global_cost, execution_time):
        """Analyse des performances"""
        final_costs = [r['final_cost'] for r in all_results]
        
        print(f"\n{'='*50}")
        print(f"📊 RÉSULTATS FINAUX")
        print(f"{'='*50}")
        print(f"Meilleure solution: {best_global_cost:.2f}")
        print(f"Coût moyen: {np.mean(final_costs):.2f} ± {np.std(final_costs):.2f}")
        print(f"Plage: {min(final_costs):.2f} - {max(final_costs):.2f}")
        print(f"Temps: {execution_time:.2f}s")

# Utilisation simple et directe
def main():
    random.seed(42)
    
    # Matrice réaliste
    n = 20
    points = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(n)]
    distance_matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j:
                dx = points[i][0] - points[j][0]
                dy = points[i][1] - points[j][1]
                distance_matrix[i][j] = np.sqrt(dx*dx + dy*dy)
    
    # MULTI-START TABU SEARCH en une ligne !
    solver = MultiStartTabuSearch(
        distance_matrix, 
        num_restarts=10, 
        tabu_size=20, 
        max_iterations=300
    )
    
    meilleure_solution, meilleur_cout = solver.solve()
    
    print(f"\n🏆 Solution optimale: {meilleur_cout:.2f}")
    print(f"Tournée: {' → '.join(map(str, meilleure_solution))}")

if __name__ == "__main__":
    main()

🚀 MULTI-START TABU SEARCH - 10 relances

Relance  1:  1180.88 →   361.68 (Δ:  819.19)
Relance  2:  1046.19 →   361.68 (Δ:  684.51)
Relance  3:   942.24 →   361.68 (Δ:  580.56)
Relance  4:   981.96 →   382.21 (Δ:  599.75)
Relance  5:  1017.00 →   352.32 (Δ:  664.68)
Relance  6:  1212.76 →   352.32 (Δ:  860.44)
Relance  7:   984.07 →   360.15 (Δ:  623.92)
Relance  8:  1044.88 →   406.24 (Δ:  638.65)
Relance  9:  1054.75 →   352.32 (Δ:  702.43)
Relance 10:  1106.74 →   361.68 (Δ:  745.06)

📊 RÉSULTATS FINAUX
Meilleure solution: 352.32
Coût moyen: 365.23 ± 15.96
Plage: 352.32 - 406.24
Temps: 1.76s

🏆 Solution optimale: 352.32
Tournée: 13 → 1 → 11 → 4 → 0 → 9 → 3 → 12 → 19 → 14 → 18 → 10 → 15 → 2 → 7 → 16 → 17 → 8 → 5 → 6 → 13
